In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from functools import partial
from multiprocessing import shared_memory
from multiprocessing.dummy import Pool
from sklearn.ensemble import RandomForestRegressor
import multiprocessing as mp
from itertools import chain, combinations

import mp_run

In [ ]:
perturbation_factor = 3
num_rf_predictors = 500

In [ ]:
tf_df = pd.read_csv('data/Ath_TF_list.txt', sep='\t')
tf_list = tf_df['Gene_ID']

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [ ]:
deg_df = pd.read_csv('data/SS/92_DEG_Clusters.csv', index_col=0)
deg_genes = deg_df.index

In [ ]:
# deg genes for presentation only:
deg_genes = ['AT1G77760', 'AT3G44300', 'AT1G69870', 'AT2G46680', 'AT2G46820', 'AT1G14040']

In [40]:
ts_df = pd.read_csv('data/GSE111062_RAW/expression.tsv', sep='\t', index_col=0)

In [41]:
target_genes = pd.Series(list(set(deg_genes).intersection(set(ts_df.index))))

In [42]:
meta_df = pd.read_csv('data/GSE111062_RAW/meta_data.tsv', sep='\t')
ts_exp_index = meta_df[meta_df['isTs']]
# ts_exp_index_target =  ts_exp_index[ts_exp_index['is1stLast'] != 'f'].condName
# ts_exp_index_source =  ts_exp_index[ts_exp_index['is1stLast'] != 'f'].prevCol
ts_exp_index_target =  ts_exp_index.condName
ts_exp_index_source =  ts_exp_index.condName
regulator_gene_index = ts_df.index
regulator_gene_index = pd.Series(list(set(tf_list).intersection(set(regulator_gene_index))))


In [43]:
importance_df_list = []

In [44]:

regr = RandomForestRegressor(random_state=0)


ts_train_y_list = ts_df[ts_exp_index_target]

result_list = []
result_measure_list = []

for target_gene in tqdm(target_genes):

    train_gene_index = regulator_gene_index[regulator_gene_index != target_gene]
    ts_train_X = ts_df[ts_exp_index_source].T[train_gene_index]

    ts_train_y = ts_train_y_list.loc[target_gene]

    input_mean = ts_train_X.mean()
    input_std = ts_train_X.std()
    perturbation_input = input_mean.copy()
    perturbation_input = np.repeat(np.array(input_mean), repeats=len(input_mean), axis=0)
    perturbation_input = perturbation_input.reshape(len(input_mean),len(input_mean)) + np.diagflat(np.array([ts_train_X.values.std()]*len(input_mean)) * perturbation_factor)

    func = partial(mp_run.rf_feature_importance, ts_train_X, ts_train_y)

    with mp.Pool() as pool:
        results = pool.map(func, range(num_rf_predictors))

    base_predictions = [result.predict(np.array(input_mean).reshape(1,-1))[0] for result in results]
    y_std = ts_train_y.std()

    perturbation_input_list = np.array_split(perturbation_input, 10)
    perturbation_input_shapes = [p.shape for p in perturbation_input_list]
    shared_memory_list = []
    for i, p in enumerate(perturbation_input_list):
        shm = shared_memory.SharedMemory(create=True, size=p.nbytes, name="perturbation_input_"+str(i))
        buffer = np.ndarray(p.shape, dtype=p.dtype, buffer=shm.buf)
        buffer[:] = p[:]
        shared_memory_list.append(shm)


    def f(perturbation_input_shapes, x):
        pred_list = []
        for i, shape in enumerate(perturbation_input_shapes):

            # Attach to the existing shared memory
            existing_shm = shared_memory.SharedMemory(name='perturbation_input_'+str(i))
            # Read from the shared memory (we know the size is 1)
            c = np.ndarray(shape, dtype=np.float64, buffer=existing_shm.buf)
            pred_list.append(x.predict(c))
            existing_shm.close()
        return np.concatenate(pred_list)

    func = partial(f, perturbation_input_shapes)

    with mp.Pool(processes=20) as pool:
        perturbation_predictions = pool.map(func, results)

    for shm in shared_memory_list:
        shm.close()
        shm.unlink()


    perturbation_measures = [(perturbation_prediction - base_prediction)/y_std for perturbation_prediction, base_prediction in zip(perturbation_predictions, base_predictions)]
    importance_matrix = np.array(perturbation_measures).T
    importance_df = pd.DataFrame(index=train_gene_index, data=importance_matrix, columns=range(num_rf_predictors))
    importance_df_list
    mean_importance = importance_df.mean(axis=1)
    top_influence_genes = train_gene_index[np.argsort(mean_importance)[-5:]]
    importance_df_list.append(mean_importance)
    data_mean = ts_df.T[top_influence_genes].mean()
    data_std = ts_df.T[top_influence_genes].std()
    regr = RandomForestRegressor(random_state=42, warm_start=True, n_estimators=100, n_jobs=20)
    ts_train_X = ts_df[ts_exp_index_source].T[top_influence_genes]
    regr = regr.fit(ts_train_X, ts_train_y)

    base_prediction = regr.predict(np.array(data_mean).reshape(1,-1))[0]
    y_std = ts_df.T.std()[target_gene]
    perturbation_list = list(powerset(top_influence_genes))[1:]

    perturbation_result_list = []
    perturbation_list_names = ['; '.join(perturbation_genes) for perturbation_genes in perturbation_list]
    for perturbation_genes in perturbation_list:
        perturbation_input = data_mean.copy()
        for gene in perturbation_genes:
            perturbation_input[gene] += data_std[gene] * perturbation_factor
        perturbation_prediction = regr.predict(np.array(perturbation_input).reshape(1,-1))[0]
        perturbation_measure = (perturbation_prediction - base_prediction)/y_std
        perturbation_result_list.append(perturbation_measure)
    result_list.append(np.array(perturbation_list_names)[np.argsort(perturbation_result_list)[::-1][:5]])
    result_measure_list.append(np.array(perturbation_result_list)[np.argsort(perturbation_result_list)[::-1][:5]])





100%|██████████| 6/6 [04:41<00:00, 46.89s/it]


In [ ]:
result_measure_list = np.array(result_measure_list)
result_list = np.array(result_list)
out_df = pd.DataFrame()
out_df.index = target_genes
for i in range(5):
    comb_name = 'top_{}_combination'.format(i+1)
    score_name = 'top_{}_score'.format(i+1)
    out_df[comb_name] = result_list[:,i]
    out_df[score_name] = result_measure_list[:,i]

In [ ]:
out_df.to_csv('output/GSE111062/presentation_comb.csv')

In [45]:
from email import header


for importance_series, target_gene in zip(importance_df_list, target_genes):
    importance_series.to_csv('output/GSE111062/'+target_gene+'_rankings.csv', header=False)

In [ ]:
importance_df_list[5].sort_values().index.get_loc('AT3G49690')

In [ ]:
importance_df_list[5].sort_values().index.get_loc('AT3G26744')

In [ ]:
importance_df_list[5].sort_values().index.get_loc('AT3G23250')

In [ ]:
importance_df_list[5].sort_values().index.get_loc('AT3G26744')